In [3]:
# === PASO 1: Limpieza y combinación de bases ===
import pandas as pd

# Cargar archivos CSV
clientes = pd.read_csv('base_clientes_final.csv')
transacciones = pd.read_csv('base_transacciones_final.csv')

# Limpiar nombres de columnas por si acaso
clientes.rename(columns=lambda x: x.strip(), inplace=True)
transacciones.rename(columns=lambda x: x.strip(), inplace=True)

# Combinar las bases usando 'id',,,
base_completa = transacciones.merge(clientes, on='id', how='left')

# Llenar nulos en giro_comercio
base_completa['giro_comercio'].fillna("SIN CLASIFICAR", inplace=True)

# Vista previa
print("Vista previa de la base combinada:")
print(base_completa.head())

# Guardar base combinada
base_completa.to_csv('base_completa.csv', index=False)
print("\n¡Archivo guardado como 'base_completa.csv'!")


/var/folders/22/pgty7krs2sldhc8zl8xgk9pm0000gn/T/ipykernel_54401/3211244414.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  base_completa['giro_comercio'].fillna("SIN CLASIFICAR", inplace=True)


Vista previa de la base combinada:
                                         id       fecha comercio  \
0  91477f382c3cf63ab5cd9263b502109243741158  2022-01-02   AMAZON   
1  91477f382c3cf63ab5cd9263b502109243741158  2022-01-05    RAPPI   
2  91477f382c3cf63ab5cd9263b502109243741158  2022-01-05    RAPPI   
3  91477f382c3cf63ab5cd9263b502109243741158  2022-01-05   AMAZON   
4  91477f382c3cf63ab5cd9263b502109243741158  2022-01-05   AMAZON   

                                giro_comercio tipo_venta  monto  \
0  COMERCIOS ELECTRONICOS (VTAS POR INTERNET)    digital   5.99   
1   SERVICIOS EMPRESARIALES - NO CLASIFICADOS    digital  13.01   
2   SERVICIOS EMPRESARIALES - NO CLASIFICADOS    digital  15.84   
3  COMERCIOS ELECTRONICOS (VTAS POR INTERNET)    digital   8.17   
4  COMERCIOS ELECTRONICOS (VTAS POR INTERNET)    digital   2.54   

  fecha_nacimiento  fecha_alta  id_municipio  id_estado  \
0       1987-09-21  2019-10-28       6519019         65   
1       1987-09-21  2019-10-28     

In [5]:
# PASO 2: Calcular meses_distintos por cliente–comercio
# Asegurarse que la fecha sea tipo datetime
base_completa['fecha'] = pd.to_datetime(base_completa['fecha'])

# Crear columna año-mes
base_completa['año_mes'] = base_completa['fecha'].dt.to_period('M')

# Agrupar por cliente y comercio para contar meses distintos
frecuencia_mensual = (
    base_completa.groupby(['id', 'comercio'])['año_mes']
    .nunique()
    .reset_index(name='meses_distintos')
)


In [ ]:
# === PASO 2.5: Enriquecer con datos del cliente ===

info_clientes = base_completa[['id', 'fecha_nacimiento', 'fecha_alta', 'id_municipio', 'id_estado',
                               'tipo_persona', 'genero', 'actividad_empresarial']].drop_duplicates()

info_clientes['fecha_nacimiento'] = pd.to_datetime(info_clientes['fecha_nacimiento'])
info_clientes['fecha_alta'] = pd.to_datetime(info_clientes['fecha_alta'])

hoy = pd.to_datetime('today')
info_clientes['edad'] = (hoy - info_clientes['fecha_nacimiento']).dt.days // 365
info_clientes['antiguedad_meses'] = (hoy - info_clientes['fecha_alta']).dt.days // 30

# Unir con la tabla recurrentes
recurrentes_full = recurrentes.merge(info_clientes, on='id', how='left')
print("\nTabla lista para entrenar modelo:")
print(recurrentes_full.head())


In [ ]:
# === PASO 3: Entrenar modelo de clasificación ===
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

features = ['num_transacciones', 'monto_prom', 'intervalo_prom', 'edad', 'antiguedad_meses']
X = recurrentes_full[features]
y = recurrentes_full['es_recurrente']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print("\nReporte de clasificación:")
print(classification_report(y_test, y_pred))
